# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [3]:
# # insider db
# wb = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# exasol db
ws = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYS')
wb = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# fold path

In [4]:
drive = 'D'
ods_fold = f'{drive}:/data/big2/ODS'
big2_fold = f'{drive}:/data/big2'
an3_fold = f'{drive}:/data/big2/BD3/an'
raw3_fold = f'{drive}:/data/big2/BD3/raw'

# Load
- 등록&제원 -> result 5등급

## 등록정보(STD_CEG_CAR_MIG) 5등급만

In [5]:
# 8.6s
car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0505' OR EXHST_GAS_GRD_CD = 'A05T5';")

C:\Users\kbjung\AppData\Roaming\Python\Python39\site-packages\pyexasol\callback.py:42: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(pipe, skip_blank_lines=False, **kwargs)


In [6]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2378646 entries, 0 to 2378645
Data columns (total 10 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   VIN             object
 1   BSPL_STDG_CD    int64 
 2   VHCL_ERSR_YN    object
 3   MANP_MNG_NO     object
 4   YRIDNW          int64 
 5   VHCTY_CD        object
 6   PURPS_CD2       object
 7   FRST_REG_YMD    int64 
 8   VHCL_FBCTN_YMD  object
 9   VHCL_MNG_NO     object
dtypes: int64(3), object(7)
memory usage: 181.5+ MB


In [7]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호',
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
    'VHCL_MNG_NO':'차량관리번호'
}

In [8]:
carr = car.rename(columns=car_ch_col)

In [9]:
carr.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [10]:
# 3.8s
src = wb.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2 FROM STD_CEG_CAR_SRC_MIG;")

In [11]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
}

In [12]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형'], dtype='object')

## 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

In [13]:
# 3.0s
att = wb.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010946 entries, 0 to 1010945
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   VIN           1010946 non-null  object
 1   RDCDVC_SE_CD  1010946 non-null  object
dtypes: object(2)
memory usage: 15.4+ MB


In [14]:
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분', 
}

In [15]:
attr = att.rename(columns=att_ch_col)
attr.columns

Index(['차대번호', '저감장치구분'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [16]:
# 1.3s
code = wb.export_to_pandas("SELECT STDG_CD, STDG_CTPV_CD, STDG_SGG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46333 entries, 0 to 46332
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       46333 non-null  int64 
 1   STDG_CTPV_CD  46333 non-null  int64 
 2   STDG_SGG_CD   46333 non-null  int64 
 3   STDG_CTPV_NM  46333 non-null  object
 4   STDG_SGG_NM   46310 non-null  object
dtypes: int64(3), object(2)
memory usage: 1.8+ MB


In [17]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_CD', 'STDG_SGG_CD', 'STDG_CTPV_NM',
       'STDG_SGG_NM'],
      dtype='object')

In [18]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    'STDG_CTPV_CD':'시도코드', 
    'STDG_SGG_CD':'시군구코드',
}

In [19]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도코드', '시군구코드', '시도', '시군구'], dtype='object')

## 노후차 조기폐차 관리정보(수도권)(STD_DLM_TB_ERP_EARLY_ERASE_AEA)

In [20]:
# 2.4s
aea = wb.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN FROM STD_DLM_TB_ERP_EARLY_ERASE_AEA;")
aea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804082 entries, 0 to 804081
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   VIN                  804082 non-null  object
 1   ELPDSRC_STTS_CD      804082 non-null  object
 2   ELPDSRC_LST_APRV_YN  683874 non-null  object
dtypes: object(3)
memory usage: 18.4+ MB


In [21]:
aea.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'], dtype='object')

In [22]:
aea_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN',  
}

In [23]:
aear = aea.rename(columns=aea_ch_col)
aear.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN'], dtype='object')

In [24]:
aear.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN
0,KMCXLS7BP2U626435,A32Y,Y
1,KNAJA5265VA794691,A32Y,Y
2,KPBLC3D81VP018295,A32Y,Y
3,KPDHADMD1YP225493,A32Y,Y
4,KNAMA75122S249083,A32Y,Y


## 노후차 조기폐차 관리정보(수도권외)(STD_DLM_TB_ERP_EARLY_ERASE_LGV)

In [25]:
# 1.8s
lgv = wb.export_to_pandas("SELECT VIN, ELPDSRC_STTS_CD, ELPDSRC_LST_APRV_YN FROM STD_DLM_TB_ERP_EARLY_ERASE_LGV;")
lgv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373371 entries, 0 to 373370
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   VIN                  373371 non-null  object
 1   ELPDSRC_STTS_CD      373371 non-null  object
 2   ELPDSRC_LST_APRV_YN  160690 non-null  object
dtypes: object(3)
memory usage: 8.5+ MB


In [26]:
lgv.columns

Index(['VIN', 'ELPDSRC_STTS_CD', 'ELPDSRC_LST_APRV_YN'], dtype='object')

In [27]:
lgv_ch_col = {
    'VIN':'차대번호', 
    'ELPDSRC_STTS_CD':'조기폐차상태코드', 
    'ELPDSRC_LST_APRV_YN':'조기폐차최종승인YN',  
}

In [28]:
lgvr = lgv.rename(columns=lgv_ch_col)
lgvr.columns

Index(['차대번호', '조기폐차상태코드', '조기폐차최종승인YN'], dtype='object')

In [29]:
lgvr.head()

,차대번호,조기폐차상태코드,조기폐차최종승인YN
0,KMHSD81VP4U791871,A32M,NaN
1,KPBFA2AF16P223800,A32M,NaN
2,KMFWVH7HP7U772891,A32M,NaN
3,KMFXKN7BPYU361960,A32M,NaN
4,KMFXKS7BP2U563724,A32M,NaN


## 비상시 및 계절제 단속발령(N_IS_ISSUE_DISCLOSURE)

In [ ]:
# # 1.8s
# isdis = we.export_to_pandas("SELECT GNFD_NO, VIN, REG_SIDO_CD, REG_SIGNGU_CD FROM ODS_N_IS_ISSUE_DISCLOSURE;")
# isdis.info()

In [98]:
# about 2.1s
name = 'ODS_N_IS_ISSUE_DISCLOSURE(col_sliced)'
file_name = f'{name}.csv'
isdis = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['REGLT_NO', 'GNFD_NO', 'VIN', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'REGLT_AREA_CD'], nrows=100**2) # usecols=['REGLT_NO', 'GNFD_NO, 'VIN', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'REGLT_AREA_CD']
isdis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   REGLT_NO       10000 non-null  int64 
 1   GNFD_NO        10000 non-null  int64 
 2   REG_SIDO_CD    10000 non-null  int64 
 3   REG_SIGNGU_CD  10000 non-null  int64 
 4   REGLT_AREA_CD  10000 non-null  int64 
 5   VIN            10000 non-null  object
dtypes: int64(5), object(1)
memory usage: 468.9+ KB


In [99]:
isdis.head()

,REGLT_NO,GNFD_NO,REG_SIDO_CD,REG_SIGNGU_CD,REGLT_AREA_CD,VIN
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943
1,54793839,1838,41,190,41190,KNAMA75432S388470
2,54793843,1838,28,237,41190,KNGDNM9N14K178854
3,54795717,1838,41,130,41190,KMFGA17PP7C072098
4,54793837,1838,41,190,41190,KPBSH3AB18P082348


In [100]:
isdis.columns

Index(['REGLT_NO', 'GNFD_NO', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'REGLT_AREA_CD',
       'VIN'],
      dtype='object')

In [101]:
isdis_ch_col = {
    'REGLT_NO':'적발번호', 
    'GNFD_NO':'발령번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'VIN':'차대번호',
    'REGLT_AREA_CD':'적발지역코드', 
}

In [102]:
isdisr = isdis.rename(columns=isdis_ch_col)
isdisr.columns

Index(['적발번호', '발령번호', '등록시도코드', '등록시군구코드', '적발지역코드', '차대번호'], dtype='object')

## 운행제한 발령정보(N_IS_ISSUE)

In [35]:
# # 1.8s
# isis = we.export_to_pandas("SELECT GNFD_NO, TY_STDR_ID, DNSTY_STDR_ID FROM ODS_N_IS_ISSUE;")
# isis.info()

In [36]:
name = 'ODS_N_IS_ISSUE'
file_name = f'{name}.csv'
isis = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['GNFD_NO', 'TY_STDR_ID', 'DNSTY_STDR_ID']) # usecols=['GNFD_NO', 'TY_STDR_ID', 'DNSTY_STDR_ID']
isis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1785 entries, 0 to 1784
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   GNFD_NO        1785 non-null   int64 
 1   DNSTY_STDR_ID  1785 non-null   object
 2   TY_STDR_ID     1785 non-null   object
dtypes: int64(1), object(2)
memory usage: 42.0+ KB


In [37]:
isis.columns

Index(['GNFD_NO', 'DNSTY_STDR_ID', 'TY_STDR_ID'], dtype='object')

In [38]:
isis_ch_col = {
    'GNFD_NO':'발령번호', 
    'DNSTY_STDR_ID':'농도기준아이디', 
    'TY_STDR_ID':'유형기준아이디', 
}

In [39]:
isisr = isis.rename(columns=isis_ch_col)
isisr.columns

Index(['발령번호', '농도기준아이디', '유형기준아이디'], dtype='object')

In [40]:
# DNSTY_STDR_ID(농도기준아이디) : 실발령(C011), 모의발령(C012)
# TY_STDR_ID(유형기준아이디) : 비상시(T001), 계절제(T002)

## N_IS_PENALTY

In [41]:
# 1.8s
ispe = ws.export_to_pandas("SELECT REGLT_NO, REGLT_DE FROM N_IS_PENALTY WHERE ROWNUM <= 10000;")
# ispe = ws.export_to_pandas("SELECT REGLT_NO, REGLT_DE FROM N_IS_PENALTY;")
ispe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   REGLT_NO  10000 non-null  int64
 1   REGLT_DE  10000 non-null  int64
dtypes: int64(2)
memory usage: 156.4 KB


In [151]:
# # about 2.1s
# name = 'N_IS_PENALTY'
# file_name = f'{name}.csv'
# ispe = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['REGLT_NO, 'REGLT_DE'], nrows=100**2, sep='|') # usecols=['REGLT_NO, 'REGLT_DE']
# ispe.info()

In [42]:
ispe.head()

,REGLT_NO,REGLT_DE
0,40995,20191210
1,40828,20191210
2,40676,20191210
3,40683,20191210
4,42850,20191210


In [43]:
ispe.columns

Index(['REGLT_NO', 'REGLT_DE'], dtype='object')

In [44]:
ispe_ch_col = {
    'REGLT_NO':'적발번호', 
    'REGLT_DE':'단속일', 
}

In [45]:
isper = ispe.rename(columns=ispe_ch_col)
isper.columns

Index(['적발번호', '단속일'], dtype='object')

In [46]:
isper.head()

,적발번호,단속일
0,40995,20191210
1,40828,20191210
2,40676,20191210
3,40683,20191210
4,42850,20191210


## 운행제한 단속정보(N_US_DISCLOSURE)

In [47]:
# # 1.8s
# ispe = we.export_to_pandas("SELECT GNFD_NO, REGLT_DE FROM N_US_DISCLOSURE WHERE ROWNUM <= 10000;")
# # ispe = we.export_to_pandas("SELECT GNFD_NO, REGLT_DE FROM N_US_DISCLOSURE;")
# ispe.info()

In [141]:
# about 
name = 'ODS_N_US_DISCLOSURE'
file_name = f'{name}.csv'
usdis = pd.read_csv(os.path.join(ods_fold, file_name), usecols=['NO', 'VIN', 'DISCL_TY', 'REGLT_AREA_CD', 'REG_SIDO_CD', 'REG_SIGNGU_CD'], nrows=100**2) # usecols=['NO', 'VIN, 'DISCL_TY', 'REGLT_AREA_CD', 'REG_SIDO_CD', 'REG_SIGNGU_CD']
usdis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   NO             10000 non-null  int64 
 1   REG_SIDO_CD    10000 non-null  int64 
 2   REG_SIGNGU_CD  10000 non-null  int64 
 3   DISCL_TY       10000 non-null  object
 4   REGLT_AREA_CD  10000 non-null  int64 
 5   VIN            10000 non-null  object
dtypes: int64(4), object(2)
memory usage: 468.9+ KB


In [142]:
usdis.head()

,NO,REG_SIDO_CD,REG_SIGNGU_CD,DISCL_TY,REGLT_AREA_CD,VIN
0,117112,41,110,S,41430,KPBFA2AD12P021974
1,117111,11,0,S,41430,KMHNN81XP2U038507
2,117107,41,410,S,41430,KMFZCD7JP5U094049
3,132833,11,0,S,41130,KMFWVH7HP4U579325
4,6933349,11,0,S,41130,KNAMA75433S411400


In [143]:
usdis.columns

Index(['NO', 'REG_SIDO_CD', 'REG_SIGNGU_CD', 'DISCL_TY', 'REGLT_AREA_CD',
       'VIN'],
      dtype='object')

In [144]:
usdis_ch_dict = {
    'NO':'번호', 
    'VIN':'차대번호', 
    'REG_SIDO_CD':'등록시도코드', 
    'REG_SIGNGU_CD':'등록시군구코드', 
    'DISCL_TY':'적발유형', 
    'REGLT_AREA_CD':'단속지역코드', 
}

In [145]:
usdisr = usdis.rename(columns=usdis_ch_dict)
usdisr.columns

Index(['번호', '등록시도코드', '등록시군구코드', '적발유형', '단속지역코드', '차대번호'], dtype='object')

In [53]:
# DISCL_TY(적발유형) : S(일반), H(고농도->상시)

## N_US_PENALTY

In [54]:
# 1.8s
uspe = ws.export_to_pandas('SELECT "NO", REGLT_CNT FROM N_US_PENALTY;')
uspe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443309 entries, 0 to 443308
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   NO         443309 non-null  int64
 1   REGLT_CNT  443309 non-null  int64
dtypes: int64(2)
memory usage: 6.8 MB


In [55]:
# # about 2.1s
# name = 'N_US_PENALTY'
# file_name = f'{name}.csv'
# uspe = pd.read_csv(os.path.join(ods_fold, file_name), nrows=100**2, sep='|' ) # usecols=['NO', 'REGLT_CNT']
# uspe.info()

In [56]:
uspe.head()

,NO,REGLT_CNT
0,8114417,1
1,6933701,4
2,6938411,10
3,8114173,1
4,6938670,14


In [57]:
uspe.columns

Index(['NO', 'REGLT_CNT'], dtype='object')

In [58]:
uspe_ch_dict = {
    'NO':'번호', 
    'REGLT_CNT':'적발건수', 
}

In [59]:
usper = uspe.rename(columns=uspe_ch_dict)
usper.columns

Index(['번호', '적발건수'], dtype='object')

# 전처리

## 중복 차대번호 제거

In [60]:
carr.shape, len(carr['차대번호'].unique())

((2378646, 10), 2378598)

In [61]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(2378598, 10)

## 차종 코드 변환

In [62]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['화물', '승용', '특수', '승합', nan], dtype=object)

## 용도 코드 변환

In [63]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [64]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1162311, 10)

## 연료 코드 변환

In [65]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '알코올', '전기', '태양열', 'CNG 하이브리드', '경유 하이브리드', '등유',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [66]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [67]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1162311, 12)

## 저감장치구분 코드 변환

In [68]:
# 4m 14.3s
attr['저감장치구분'].unique()

array(['A1001', 'A1003', 'A1005', 'A1006', 'A1002', 'A1004', 'A1007'],
      dtype=object)

In [69]:
# 27.5s
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['1종', '3종', '엔진개조', '엔진교체', '2종', '1종+SCR', '삼원촉매'], dtype=object)

## 저감장치 부착 유무

In [70]:
attr.loc[(attr['저감장치구분'] == '1종') | (attr['저감장치구분'] == '1종+SCR'), 'DPF_YN'] = '유'

In [71]:
attr['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    531833
유      479113
Name: count, dtype: int64

## 등록&제원&저감이력 병합

In [72]:
# 1.7s
csa = cs.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
csa.shape

(1162570, 13)

## 지역정보 병합

In [73]:
csac = csa.merge(coder, on='법정동코드', how='left')

### 매칭 안되는 지역 처리

In [74]:
csac.loc[csac['법정동코드'] == 4165052000, ['시도', '시군구']] = ['경기도', '포천시'] # 경기도 포천시 선단동

## 조기폐차 정보들 병합

In [75]:
elp = pd.concat([aear, lgvr], ignore_index=True)
elp.shape

(1177453, 3)

In [76]:
elp.shape, len(elp['차대번호'].unique())

((1177453, 3), 1105195)

In [77]:
elpm = elp.sort_values('조기폐차최종승인YN', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
elpm.shape

(1105195, 3)

In [78]:
elpm = elpm[elpm['조기폐차최종승인YN'] == 'Y'].reset_index(drop=True)
elpm.shape

(844474, 3)

## 조기폐차 정보 추가

In [79]:
csac.shape

(1162570, 17)

In [80]:
df = csac.merge(elpm, on='차대번호', how='left')
df.shape

(1162570, 19)

## 비상시 및 계절제 병합

In [103]:
isdisr.shape, isisr.shape, isper.shape

((10000, 6), (1785, 3), (10000, 2))

In [104]:
isdisr.columns, isisr.columns, isper.columns

(Index(['적발번호', '발령번호', '등록시도코드', '등록시군구코드', '적발지역코드', '차대번호'], dtype='object'),
 Index(['발령번호', '농도기준아이디', '유형기준아이디'], dtype='object'),
 Index(['적발번호', '단속일'], dtype='object'))

In [105]:
isdisr.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943
1,54793839,1838,41,190,41190,KNAMA75432S388470
2,54793843,1838,28,237,41190,KNGDNM9N14K178854
3,54795717,1838,41,130,41190,KMFGA17PP7C072098
4,54793837,1838,41,190,41190,KPBSH3AB18P082348


In [106]:
isdisr.shape

(10000, 6)

In [107]:
isdp= isdisr.merge(isper, on='적발번호', how='left')
isdp.shape

(10000, 7)

In [108]:
isdp.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN


In [109]:
isdpi = isdp.merge(isisr, on='발령번호', how='left')
isdpi.shape

(10000, 9)

In [110]:
isdpi.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN,C012,T001
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN,C012,T001
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN,C012,T001
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN,C012,T001
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN,C012,T001


### 지역정보 추가

In [111]:
coder.head()

,법정동코드,시도코드,시군구코드,시도,시군구
0,4171004506,41,710,경기도,양주군
1,4171004507,41,710,경기도,양주군
2,4171004508,41,710,경기도,양주군
3,4171004509,41,710,경기도,양주군
4,4171005000,41,710,경기도,양주군


In [112]:
coder_dup = coder.sort_values('법정동코드', ascending=True).drop_duplicates(['시도코드', '시군구코드']).reset_index(drop=True)
coder_dup.head()

,법정동코드,시도코드,시군구코드,시도,시군구
0,1100000000,11,0,서울특별시,NaN
1,1111000000,11,110,서울특별시,종로구
2,1114000000,11,140,서울특별시,중구
3,1117000000,11,170,서울특별시,용산구
4,1120000000,11,200,서울특별시,성동구


In [130]:
is_total1 = isdpi.merge(coder_dup[['시도코드', '시군구코드', '시도', '시군구']], left_on=['등록시도코드', '등록시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
is_total1 = is_total1.drop(['시도코드', '시군구코드'], axis=1)
is_total1.shape

(10000, 11)

In [131]:
is_total1.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,시도,시군구
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN,C012,T001,경기도,부천시
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN,C012,T001,경기도,부천시
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN,C012,T001,인천광역시,부평구
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN,C012,T001,경기도,성남시
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN,C012,T001,경기도,부천시


In [132]:
is_total1 = is_total1.rename(columns={'시도':'등록시도', '시군구':'등록시군구'})
is_total1.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN,C012,T001,경기도,부천시
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN,C012,T001,경기도,부천시
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN,C012,T001,인천광역시,부평구
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN,C012,T001,경기도,성남시
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN,C012,T001,경기도,부천시


In [133]:
is_total1['적발지역코드'] = is_total1['적발지역코드'].astype('str')
is_total1['적발시도코드'] = is_total1['적발지역코드'].str[:2]
is_total1['적발시군구코드'] = is_total1['적발지역코드'].str[2:5]
is_total1[['적발시도코드', '적발시군구코드']] = is_total1[['적발시도코드', '적발시군구코드']].astype('int')
is_total1.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구,적발시도코드,적발시군구코드
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN,C012,T001,경기도,부천시,41,190
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN,C012,T001,경기도,부천시,41,190
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN,C012,T001,인천광역시,부평구,41,190
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN,C012,T001,경기도,성남시,41,190
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN,C012,T001,경기도,부천시,41,190


In [134]:
is_total = is_total1.merge(coder_dup[['시도코드', '시군구코드', '시도', '시군구']], left_on=['적발시도코드', '적발시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
is_total.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구,적발시도코드,적발시군구코드,시도코드,시군구코드,시도,시군구
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN,C012,T001,경기도,부천시,41,190,41.00,190.00,경기도,부천시
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN,C012,T001,경기도,부천시,41,190,41.00,190.00,경기도,부천시
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN,C012,T001,인천광역시,부평구,41,190,41.00,190.00,경기도,부천시
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN,C012,T001,경기도,성남시,41,190,41.00,190.00,경기도,부천시
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN,C012,T001,경기도,부천시,41,190,41.00,190.00,경기도,부천시


In [135]:
is_total = is_total.drop(['적발시도코드', '적발시군구코드', '시도코드', '시군구코드'], axis=1)
is_total = is_total.rename(columns={'시도':'적발시도', '시군구':'적발시도'})
is_total.head()

,적발번호,발령번호,등록시도코드,등록시군구코드,적발지역코드,차대번호,단속일,농도기준아이디,유형기준아이디,등록시도,등록시군구,적발시도,적발시도
0,54793831,1838,41,190,41190,KMFXKS7BP3U657943,NaN,C012,T001,경기도,부천시,경기도,부천시
1,54793839,1838,41,190,41190,KNAMA75432S388470,NaN,C012,T001,경기도,부천시,경기도,부천시
2,54793843,1838,28,237,41190,KNGDNM9N14K178854,NaN,C012,T001,인천광역시,부평구,경기도,부천시
3,54795717,1838,41,130,41190,KMFGA17PP7C072098,NaN,C012,T001,경기도,성남시,경기도,부천시
4,54793837,1838,41,190,41190,KPBSH3AB18P082348,NaN,C012,T001,경기도,부천시,경기도,부천시


## 상시 병합

In [146]:
usdisr.head()

,번호,등록시도코드,등록시군구코드,적발유형,단속지역코드,차대번호
0,117112,41,110,S,41430,KPBFA2AD12P021974
1,117111,11,0,S,41430,KMHNN81XP2U038507
2,117107,41,410,S,41430,KMFZCD7JP5U094049
3,132833,11,0,S,41130,KMFWVH7HP4U579325
4,6933349,11,0,S,41130,KNAMA75433S411400


In [147]:
usper.head()

,번호,적발건수
0,8114417,1
1,6933701,4
2,6938411,10
3,8114173,1
4,6938670,14


In [148]:
usdisr.shape

(10000, 6)

In [149]:
usdp = usdisr.merge(usper, on='번호', how='left')
usdp.shape

(10000, 7)

### 지역정보 추가

In [150]:
us_total = usdp.merge(coder_dup, left_on=['등록시도코드', '등록시군구코드'], right_on=['시도코드', '시군구코드'], how='left')
us_total.head()

,번호,등록시도코드,등록시군구코드,적발유형,단속지역코드,차대번호,적발건수,법정동코드,시도코드,시군구코드,시도,시군구
0,117112,41,110,S,41430,KPBFA2AD12P021974,NaN,4111000000,41,110,경기도,수원시
1,117111,11,0,S,41430,KMHNN81XP2U038507,NaN,1100000000,11,0,서울특별시,NaN
2,117107,41,410,S,41430,KMFZCD7JP5U094049,NaN,4141000000,41,410,경기도,군포시
3,132833,11,0,S,41130,KMFWVH7HP4U579325,NaN,1100000000,11,0,서울특별시,NaN
4,6933349,11,0,S,41130,KNAMA75433S411400,NaN,1100000000,11,0,서울특별시,NaN


In [151]:
us_total = us_total.drop(['시도코드', '시군구코드'], axis=1)
us_total.head()

,번호,등록시도코드,등록시군구코드,적발유형,단속지역코드,차대번호,적발건수,법정동코드,시도,시군구
0,117112,41,110,S,41430,KPBFA2AD12P021974,NaN,4111000000,경기도,수원시
1,117111,11,0,S,41430,KMHNN81XP2U038507,NaN,1100000000,서울특별시,NaN
2,117107,41,410,S,41430,KMFZCD7JP5U094049,NaN,4141000000,경기도,군포시
3,132833,11,0,S,41130,KMFWVH7HP4U579325,NaN,1100000000,서울특별시,NaN
4,6933349,11,0,S,41130,KNAMA75433S411400,NaN,1100000000,서울특별시,NaN


In [152]:
is_total.shape, us_total.shape

((10000, 13), (10000, 10))

# Load

## 5등급 지역별 조기폐차

In [153]:
# about 2.1s
name = '5등급_조기폐차'
file_name = f'{name}.xlsx'
df1 = pd.read_excel(os.path.join(raw3_fold, file_name))
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RGN         68 non-null     object 
 1   YR          68 non-null     int64  
 2   VHCL_MKCNT  68 non-null     int64  
 3   VHCL_REDE   68 non-null     float64
 4   ELPDSRC     68 non-null     float64
 5   NTRL_DCLN   68 non-null     float64
dtypes: float64(3), int64(2), object(1)
memory usage: 3.3+ KB


In [154]:
df1.head()

,RGN,YR,VHCL_MKCNT,VHCL_REDE,ELPDSRC,NTRL_DCLN
0,서울,2019,199898,46949.67,31792.00,15157.67
1,서울,2020,161864,38034.00,23045.00,14989.00
2,서울,2021,138381,23483.00,8148.00,15335.00
3,서울,2022,120996,17385.00,2476.00,14909.00
4,인천,2019,90619,20596.67,16421.67,4175.00


## 5등급 저공해 미조치

In [155]:
# about 2.1s
name = '5등급_저공해미조치'
file_name = f'{name}.xlsx'
no_dpf = pd.read_excel(os.path.join(raw3_fold, file_name))
no_dpf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      12 non-null     object
 1   구분      12 non-null     object
 2   차량대수    12 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 416.0+ bytes


In [156]:
no_dpf.head()

,지역,구분,차량대수
0,수도권,2차계절제_시작,539748
1,수도권,2차계절제_종료,515615
2,수도권,3차계절제_시작,459110
3,수도권,3차계절제_종료,436921
4,수도권,4차계절제_시작,394533


## [로드] 운행제한 테이블(limit)

In [23]:
# about 2.1s
name = '차대번호별 적발'
file_name = f'{name}.csv'
limit = pd.read_csv(os.path.join(raw3_fold, file_name), low_memory=False, encoding='cp949')
limit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731507 entries, 0 to 731506
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   VIN     731507 non-null  object 
 1   계절제_1차  37861 non-null   float64
 2   계절제_2차  54788 non-null   float64
 3   계절제_3차  68278 non-null   float64
 4   비상시     94084 non-null   float64
 5   상시      84185 non-null   float64
 6   계절제_4차  29307 non-null   float64
dtypes: float64(6), object(1)
memory usage: 39.1+ MB


In [24]:
limit.head()

,VIN,계절제_1차,계절제_2차,계절제_3차,비상시,상시,계절제_4차
0,KPBFA2AD14P100772,1.00,NaN,NaN,1.00,NaN,NaN
1,KMJWWH7JP5U656738,4.00,2.00,25.00,3.00,NaN,1.00
2,KNAJC52136A532787,2.00,NaN,NaN,1.00,NaN,NaN
3,KMHSH81WP6U081899,4.00,NaN,NaN,1.00,NaN,NaN
4,KNAJE55186K143510,1.00,NaN,NaN,1.00,NaN,NaN


## [로드] 계절제 적발지역별 차대번호(slimit)

In [25]:
# about 1.8s
name = '계절제 적발지역 차대번호'
file_name = f'{name}.csv'
slimit = pd.read_csv(os.path.join(raw3_fold, file_name), low_memory=False, encoding='cp949')
slimit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163294 entries, 0 to 163293
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   적발지역    163294 non-null  object 
 1   적발시도    163294 non-null  object 
 2   차대번호    163294 non-null  object 
 3   계절제_1차  39111 non-null   float64
 4   계절제_2차  58088 non-null   float64
 5   계절제_3차  73775 non-null   float64
 6   계절제_4차  31673 non-null   float64
 7   차종      162378 non-null  object 
 8   차종유형    162378 non-null  object 
dtypes: float64(4), object(5)
memory usage: 11.2+ MB


# 분석

In [162]:
us_total.head()

,번호,등록시도코드,등록시군구코드,적발유형,단속지역코드,차대번호,적발건수,법정동코드,시도,시군구
0,117112,41,110,S,41430,KPBFA2AD12P021974,NaN,4111000000,경기도,수원시
1,117111,11,0,S,41430,KMHNN81XP2U038507,NaN,1100000000,서울특별시,NaN
2,117107,41,410,S,41430,KMFZCD7JP5U094049,NaN,4141000000,경기도,군포시
3,132833,11,0,S,41130,KMFWVH7HP4U579325,NaN,1100000000,서울특별시,NaN
4,6933349,11,0,S,41130,KNAMA75433S411400,NaN,1100000000,서울특별시,NaN


In [164]:
us_total.groupby(['시도'])['적발건수'].sum()

시도
경기도     0.00
서울특별시   0.00
인천광역시   0.00
Name: 적발건수, dtype: float64

## 5등급

### 5등급 지역별 조기폐차 현황

In [9]:
STD_BD_GRD5_ELPDSRC_CURSTT = df1.copy()

#### [출력] STD_BD_GRD5_ELPDSRC_CURSTT

In [ ]:
# expdf = STD_BD_GRD5_ELPDSRC_CURSTT
# table_nm = 'STD_BD_GRD5_ELPDSRC_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 5등급 지역별 저공해미조치 차량현황

In [10]:
cdict = {
    '지역':'RGN',
    '구분':'SEASON',
    '차량대수':'VHCL_MKCNT', 
}

In [11]:
STD_BD_GRD5_LEM_N_MOD = no_dpf.rename(columns=cdict)
STD_BD_GRD5_LEM_N_MOD.columns

Index(['RGN', 'SEASON', 'VHCL_MKCNT'], dtype='object')

#### [출력] [D] STD_BD_GRD5_LEM_N_MOD

In [ ]:
# expdf = STD_BD_GRD5_LEM_N_MOD
# table_nm = 'STD_BD_GRD5_LEM_N_MOD'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 지역별 계절관리제 적발 현황

In [127]:
limit.head()

,차대번호,계절제_1차,계절제_2차,계절제_3차,비상시,상시,계절제_4차
0,KPBFA2AD14P100772,1.00,NaN,NaN,1.00,NaN,NaN
1,KMJWWH7JP5U656738,4.00,2.00,25.00,3.00,NaN,1.00
2,KNAJC52136A532787,2.00,NaN,NaN,1.00,NaN,NaN
3,KMHSH81WP6U081899,4.00,NaN,NaN,1.00,NaN,NaN
4,KNAJE55186K143510,1.00,NaN,NaN,1.00,NaN,NaN


In [128]:
limit = limit.rename(columns={'VIN':'차대번호'})

In [129]:
df.shape

(25693532, 22)

In [130]:
lmt1 = df.merge(limit, on='차대번호', how='left')
lmt1.shape

(25693532, 28)

In [131]:
lmt1.columns

Index(['차량등록번호', '본거지법정동코드', '차대번호', '제원관리번호', '배출가스인증번호', '배출가스등급', '차명',
       '연료', '차량말소YN', '차종', '차종유형', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부',
       '대기관리권역', '대기관리권역YN', '조기폐차최종승인YN', '저감장치구분', '저감장치', '계절제_1차',
       '계절제_2차', '계절제_3차', '비상시', '상시', '계절제_4차'],
      dtype='object')

In [132]:
lmt1['시도'].unique()

array(['서울특별시', '대구광역시', '인천광역시', '강원도', '경상남도', '전라북도', '충청북도', '경기도',
       '부산광역시', '대전광역시', '울산광역시', '경상북도', '제주특별자치도', '전라남도', '광주광역시',
       '충청남도', '세종특별자치시'], dtype=object)

In [133]:
lmt1.loc[(lmt1['시도'] == '서울특별시') | (lmt1['시도'] == '경기도') | (lmt1['시도'] == '인천광역시'), '지역'] = '수도권'
lmt1['지역'] = lmt1['지역'].fillna('수도권외')

In [134]:
lmt1['지역'].unique()

array(['수도권', '수도권외'], dtype=object)

In [136]:
lmt1.loc[lmt1['저감장치'] == 'DPF', '저감장치부착유무'] = '유'
lmt1['저감장치부착유무'] = lmt1['저감장치부착유무'].fillna('무')

In [137]:
lmt1.columns

Index(['차량등록번호', '본거지법정동코드', '차대번호', '제원관리번호', '배출가스인증번호', '배출가스등급', '차명',
       '연료', '차량말소YN', '차종', '차종유형', '시도', '시구군', '읍면동', '리', '지역3', '폐지여부',
       '대기관리권역', '대기관리권역YN', '조기폐차최종승인YN', '저감장치구분', '저감장치', '계절제_1차',
       '계절제_2차', '계절제_3차', '비상시', '상시', '계절제_4차', '지역', '저감장치부착유무'],
      dtype='object')

In [138]:
season = lmt1.loc[ (lmt1['계절제_1차'] > 0) | (lmt1['계절제_2차'] > 0) | (lmt1['계절제_3차'] > 0) | (lmt1['계절제_4차'] > 0),[
    '차대번호', 
    '계절제_1차', 
    '계절제_2차',
    '계절제_3차', 
    '계절제_4차',
    '지역', 
    '시도', 
    '저감장치부착유무', 
    '차종', 
    '차종유형', 
]]
season.shape

(95688, 10)

In [139]:
cdict = {
    '차대번호':'VIN', 
    '계절제_1차':'SEASON_1ODR_CRDN_NOCS', 
    '계절제_2차':'SEASON_2ODR_CRDN_NOCS',
    '계절제_3차':'SEASON_3ODR_CRDN_NOCS', 
    '계절제_4차':'SEASON_4ODR_CRDN_NOCS',
    '지역':'RGN', 
    '시도':'CTPV', 
    '저감장치부착유무':'DPF_EXTRNS_YN', 
    '차종':'VHCTY_CD', 
    '차종유형':'VHCTY_TY', 
}

In [141]:
STD_BD_SEASON_CRDN_NOCS_CURSTT = season.rename(columns=cdict)
STD_BD_SEASON_CRDN_NOCS_CURSTT.columns

Index(['VIN', 'SEASON_1ODR_CRDN_NOCS', 'SEASON_2ODR_CRDN_NOCS',
       'SEASON_3ODR_CRDN_NOCS', 'SEASON_4ODR_CRDN_NOCS', 'RGN', 'CTPV',
       'DPF_EXTRNS_YN', 'VHCTY_CD', 'VHCTY_TY'],
      dtype='object')

#### [출력] [D] STD_BD_SEASON_CRDN_NOCS_CURSTT

In [ ]:
# expdf = STD_BD_SEASON_CRDN_NOCS_CURSTT
# table_nm = 'STD_BD_SEASON_CRDN_NOCS_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 지역별 상시운행제한 단속 현황

In [ ]:
# orditm = lmt1.loc[lmt1['상시'] > 0, [
#     '차대번호',
#     '적발년도',
#     '적발건수',
#     '지역',
#     '시도',
# ]]

#### [로드]

In [145]:
# about 2.1s
name = '상시 적발지역 차대번호2'
file_name = f'{name}.csv'
orditm = pd.read_csv(os.path.join(raw3_fold, file_name), encoding='cp949')
orditm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106472 entries, 0 to 106471
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   차대번호    106472 non-null  object
 1   적발년도    106472 non-null  int64 
 2   적발건수    106472 non-null  int64 
 3   지역      106472 non-null  object
 4   시도      106472 non-null  object
dtypes: int64(2), object(3)
memory usage: 4.1+ MB


In [146]:
cdict = {
    '차대번호':'VIN', 
    '적발년도':'DSCL_YR', 
    '적발건수':'DSCL_NOCS', 
    '지역':'RGN',
    '시도':'CTPV', 
}

In [147]:
STD_BD_ORDITM_DSCL_CURSTT = orditm.rename(columns=cdict)
STD_BD_ORDITM_DSCL_CURSTT.columns

Index(['VIN', 'DSCL_YR', 'DSCL_NOCS', 'RGN', 'CTPV'], dtype='object')

#### [출력] STD_BD_ORDITM_DSCL_CURSTT

In [ ]:
# expdf = STD_BD_ORDITM_DSCL_CURSTT
# table_nm = 'STD_BD_ORDITM_DSCL_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

### 적발지역별 계절제 단속 현황 출력

In [148]:
slimit.head()

,적발지역,적발시도,차대번호,계절제_1차,계절제_2차,계절제_3차,계절제_4차,차종,차종유형
0,수도권,서울,KMFZCS7JP7U292779,4.00,NaN,NaN,NaN,화물,소형
1,수도권,인천,KMHSD81VP2U244691,1.00,3.00,NaN,NaN,승용,중형
2,수도권,인천,KMHJN81VP6U261521,1.00,NaN,NaN,NaN,승용,중형
3,수도권,인천,KNAJC52135A471475,1.00,NaN,NaN,NaN,승용,대형
4,수도권,경기,KMHSJ81WP6U029832,1.00,4.00,NaN,NaN,승용,대형


In [149]:
slimit.columns

Index(['적발지역', '적발시도', '차대번호', '계절제_1차', '계절제_2차', '계절제_3차', '계절제_4차', '차종',
       '차종유형'],
      dtype='object')

In [150]:
cdict = {
    '적발지역':'DSCL_RGN', 
    '적발시도':'DSCL_CTPV', 
    '차대번호':'VIN', 
    '계절제_1차':'SEASON_1ODR_CRDN_NOCS', 
    '계절제_2차':'SEASON_2ODR_CRDN_NOCS', 
    '계절제_3차':'SEASON_3ODR_CRDN_NOCS', 
    '계절제_4차':'SEASON_4ODR_CRDN_NOCS', 
    '차종':'VHCTY_CD',
    '차종유형':'VHCTY_TY', 
}

In [151]:
STD_BD_SEASON_DSCL_RGN_CURSTT = slimit.rename(columns=cdict)
STD_BD_SEASON_DSCL_RGN_CURSTT.columns

Index(['DSCL_RGN', 'DSCL_CTPV', 'VIN', 'SEASON_1ODR_CRDN_NOCS',
       'SEASON_2ODR_CRDN_NOCS', 'SEASON_3ODR_CRDN_NOCS',
       'SEASON_4ODR_CRDN_NOCS', 'VHCTY_CD', 'VHCTY_TY'],
      dtype='object')

#### [출력] [D] STD_BD_SEASON_DSCL_RGN_CURSTT

In [ ]:
# expdf = STD_BD_SEASON_DSCL_RGN_CURSTT
# table_nm = 'STD_BD_SEASON_DSCL_RGN_CURSTT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 5s
# wbt.import_from_pandas(expdf, table_nm)

# code end